In [46]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

# 2016_2c_parcial_1

1) Contamos con un cluster que tiene 4 computadoras. Queremos
aprovechar el paralelismo del cluster para calcular los números primos
entre 2 y 20.000.000. Para esto usaremos el conocido algoritmo de la
criba de Eratóstenes. Por ejemplo si empezamos con una lista de tipo
(2,3,4,5,6,7,8...) en un primer paso eliminamos todos los que son
mayores a 2 y divisibles por 2 y nos queda (2,3,5,7,9,11,13…) luego
eliminamos todos los mayores a 3 divisibles por 3 y nos queda
(2,3,5,7,11,13….etc) luego todos los divisibles por 5 y así
sucesivamente. El resultado final es una lista de números que son
primos. Programar este programa usando PySpark

In [47]:
def isDivisiblePor(x,y):
    return ((x%y) == 0)

data = range(2,200)
rdd = sc.parallelize(data,4)

prev = pivote = 1

pivote = rdd.reduce(lambda x,y: x if (x > pivote and x < y) else y)
while(pivote != prev):
    rdd = rdd.filter(lambda x: (pivote>=x or (x>pivote and not isDivisiblePor(x,pivote)))).cache()
    
    prev = pivote
    pivote = rdd.reduce(lambda x,y: x if (x > pivote and x < y) else y)
    
    
rdd.collect()

[2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97,
 101,
 103,
 107,
 109,
 113,
 127,
 131,
 137,
 139,
 149,
 151,
 157,
 163,
 167,
 173,
 179,
 181,
 191,
 193,
 197,
 199]

2) En este ejercicio queremos programar un sistema que recomiende
textos a usuarios en base a sus gustos sobre ciertos términos (palabras).
Se cuenta con un RDD de textos de la forma (docId, texto) donde texto
es un string de longitud variable. Además contamos con un RDD que
indica qué términos le gustan o no a cada usuario de la forma (userId,
término, score) por ejemplo (23, “calesita”, -2). Se pide programar en
Spark un programa que calcule el score total de cada documento para
cada usuario generando un RDD de la forma (userId, docId, score) en
donde el score es simplemente la suma de los scores del usuario para
los términos que aparecen en el documento. Puede haber términos en
los documentos para los cuales no exista score de algunos usuarios, en
estos casos simplemente los consideramos neutros (score=0)

In [48]:
textos = [(11,'Esto es un texto de ejemplo'),
          (12,'No tenes creatividad??'),
          (13,'No elegir textos aleatorios como lorep isum!!!!'),
          (14,'... y no se te cae ni una idea ...'),
          (15,'En el parcial se veia complicad XD'),
          (16,'Falta mucho?'),
          (17,'Ultimo texto Ultimo texto ultimo texto')]

terminos = [(1,'esto',-1),
            (1,'creatividad',2),
            (2,'esto',0),
            (2,'falta',1),
            (2,'y',-1),
            (3,'aleatorios',3),
            (3,'como',0),
            (4,'puedo',-3),
            (5,'textos',5)]

def getTermsByDoc(terms,docId):
    termsByDoc = []
    for term in terms:
        termsByDoc.append((term.lower(),docId))
    return termsByDoc

rddTextos = sc.parallelize(textos)
rddTerminos = sc.parallelize(terminos)

rddTextos = rddTextos.flatMap(lambda x: getTermsByDoc(x[1].split(),x[0]))
#[('esto', 11),
#('es', 11),
#('un', 11),
#('texto', 11),
#('de', 11),
#('ejemplo', 11),
#('no', 12),
#('tenes', 12),
#('creatividad??', 12),

rddTerminos = rddTerminos.map(lambda x: (x[1],(x[0],x[2])))
#[('esto', (1, -1)),
#('creatividad', (1, 2)),
#('esto', (2, 0)),
#('falta', (2, 1)),
#('y', (2, -1)),
#('aleatorios', (3, 3)),
#('como', (3, 0)),
#('puedo', (4, -3)),
#('textos', (5, 5))]

rddJoin = rddTextos.join(rddTerminos)
#[('falta', (16, (2, 1))),
#('y', (14, (2, -1))),
#('puedo', (17, (4, -3))),
#('como', (13, (3, 0))),
#('aleatorios', (13, (3, 3))),
#('textos', (13, (5, 5))),
#('esto', (11, (1, -1))),
#('esto', (11, (2, 0)))]

rddJoin.map(lambda x: ((x[1][1][0],x[1][0]),x[1][1][1])).reduceByKey(lambda x,y: x+y).collect()



[((2, 16), 1),
 ((1, 11), -1),
 ((2, 14), -1),
 ((5, 13), 5),
 ((2, 11), 0),
 ((3, 13), 3)]

# 2014_1c_Parcial_3op

1) Se tiene una colección de textos almacenados en un cluster.
Se quiere construir un índice invertido para la colección
completa. Programar usando Map-Reduce la construcción del
índice usando el esquema Tf-IDF

In [49]:
from math import log

textos = [(1,'Esto es un texto de ejemplo'),
          (2,'No tenes creatividad??'),
          (3,'No elegir textos aleatorios como lorep isum!!!!'),
          (4,'... y no se te cae ni una idea'),
          (5,'En el parcial se veia complicad XD pero no se si lo es'),
          (6,'Falta mucho?'),
          (7,'Ultimo texto Ultimo texto Ultimo texto')]

def getTermsByDoc(docId,tems):
    terms = []
    for term in tems:
        terms.append(((term.lower(),docId),1))
    return terms
    
rdd = sc.parallelize(textos)

N = rdd.count()

rdd.flatMap(lambda x: getTermsByDoc(x[0],x[1].split()))\
    .reduceByKey(lambda x,y: x+y).map(lambda x: (x[0][0],(x[0][1],x[1])))\
    .groupByKey()\
    .map(lambda x: (x[0],list(x[1]),round(log((N+1)/(len(list(x[1])))),2))).collect()
#el resultado es una lista de terminos (el ii) donde: 
#el primer elemento es el termino
#el segundo termino es un vector de tuplas, donde cada tupla es un id de documento y el tf de ese termino en ese documento
#el tercero es el IDF de cada termin log(N+1/tfi) donde tfi es la cantidad de docs donde aparece el termino

[('el', [(5, 1)], 2.08),
 ('si', [(5, 1)], 2.08),
 ('cae', [(4, 1)], 2.08),
 ('isum!!!!', [(3, 1)], 2.08),
 ('lo', [(5, 1)], 2.08),
 ('ni', [(4, 1)], 2.08),
 ('ultimo', [(7, 3)], 2.08),
 ('idea', [(4, 1)], 2.08),
 ('como', [(3, 1)], 2.08),
 ('falta', [(6, 1)], 2.08),
 ('...', [(4, 1)], 2.08),
 ('no', [(2, 1), (5, 1), (4, 1), (3, 1)], 0.69),
 ('y', [(4, 1)], 2.08),
 ('se', [(5, 2), (4, 1)], 1.39),
 ('textos', [(3, 1)], 2.08),
 ('aleatorios', [(3, 1)], 2.08),
 ('una', [(4, 1)], 2.08),
 ('es', [(1, 1), (5, 1)], 1.39),
 ('ejemplo', [(1, 1)], 2.08),
 ('mucho?', [(6, 1)], 2.08),
 ('esto', [(1, 1)], 2.08),
 ('pero', [(5, 1)], 2.08),
 ('en', [(5, 1)], 2.08),
 ('lorep', [(3, 1)], 2.08),
 ('xd', [(5, 1)], 2.08),
 ('elegir', [(3, 1)], 2.08),
 ('parcial', [(5, 1)], 2.08),
 ('te', [(4, 1)], 2.08),
 ('creatividad??', [(2, 1)], 2.08),
 ('de', [(1, 1)], 2.08),
 ('veia', [(5, 1)], 2.08),
 ('texto', [(7, 3), (1, 1)], 1.39),
 ('un', [(1, 1)], 2.08),
 ('complicad', [(5, 1)], 2.08),
 ('tenes', [(2, 1)], 2.